In [1]:
import torch
import numpy as np
import albumentations as A
import torch.nn.functional as F

from torch.utils.data import DataLoader
from models import Tresnet_m_ml_decoder_recipy
from dataset import CustomCombineTestDataset
from utils import set_seed, custom_combine_test_collate_fn


In [2]:
torch.cuda.empty_cache()
set_seed(31)

In [3]:
tf = A.Compose(
        [
            A.Resize(224, 224),
            A.Normalize(),
        ]
    )
valid_dataset = CustomCombineTestDataset(
        "/opt/ml/level3_cv_finalproject-cv-01/model",
        tf=tf,
    )
valid_loader = DataLoader(
        dataset=valid_dataset,
        batch_size=256,
        shuffle=True,
        num_workers=8,
        collate_fn=custom_combine_test_collate_fn,
        drop_last=False,
    )

In [4]:
model = torch.load("/opt/ml/level3_cv_finalproject-cv-01/model/save/Tresnet_m_ml_decoder_recipy_final_latest.pth")
model.cuda()
model.eval()

Tresnet_m_ml_decoder_recipy(
  (model): TResNet(
    (body): Sequential(
      (s2d): SpaceToDepth()
      (conv1): ConvNormAct(
        (conv): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): LeakyReLU(negative_slope=0.01, inplace=True)
        )
      )
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): ConvNormAct(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNormAct2d(
              64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): LeakyReLU(negative_slope=0.001, inplace=True)
            )
          )
          (conv2): ConvNormAct(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       

In [8]:
def calculate_accuracy(y_true, y_pred, class_label):
    # 해당 클래스에 대한 샘플 인덱스를 찾음
    class_idx = np.where(y_true == class_label)[0]
    
    # 해당 클래스에 대한 예측값과 실제 레이블을 비교하여 정확하게 예측된 샘플의 수 계산
    correct_predictions = np.sum(y_pred[class_idx] == y_true[class_idx])

    # 해당 클래스에 대한 정확도 계산
    accuracy = correct_predictions / len(class_idx)

    return accuracy

def calculate_error(y_true, y_pred, class_label):
    class_idx = np.where(y_true == class_label)[0]
    
    correct_predictions = np.sum(y_pred[class_idx] != y_true[class_idx])

    error = correct_predictions / len(class_idx)

    return error

In [9]:
with torch.no_grad():
    images_path, gt, pred, pos_acc, neg_acc, pos_err, neg_err = np.array([]), np.array([]), np.array([]), [], [], [], []
    flag = False
    for image_path, images, labels in valid_loader:
        images = torch.tensor(images).cuda()
        outputs = model(images)

        outputs = F.sigmoid(outputs).cpu().numpy()
        outputs = (outputs >= 0.5)
        labels = (labels >= 0.5)
        # outputs = outputs.cpu().numpy()
        if flag:
            images_path = np.concatenate((images_path, image_path), axis=0)
            gt = np.concatenate((gt, labels), axis=0)
            pred = np.concatenate((pred, outputs), axis=0)
            for l, o in zip(labels, outputs):
                pos_acc.append(calculate_accuracy(l, o, 1))
                neg_acc.append(calculate_accuracy(l, o, 0))
                pos_err.append(calculate_error(l, o, 1))
                neg_err.append(calculate_error(l, o, 0))
        else:
            images_path = image_path
            gt = labels
            pred = outputs
            for l, o in zip(labels, outputs):
                pos_acc.append(calculate_accuracy(l, o, 1))
                neg_acc.append(calculate_accuracy(l, o, 0))
                pos_err.append(calculate_error(l, o, 1))
                neg_err.append(calculate_error(l, o, 0))
            flag = True

In [11]:
print(pos_err)
print(neg_err)
print(len(pos_err))
print(len(neg_err))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:
print(pos_acc)
print(neg_acc)

In [ ]:
print(images_path.shape)
print(gt.shape)
print(pred.shape)

In [ ]:
import pandas as pd

data = {"image_path":[], "gt":[], "pred":[]}
for image_path, label, p in zip(images_path, gt, pred):
    data["image_path"].append(image_path)
    data["gt"].append(label)
    data["pred"].append(p)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
filename = 'output_true.csv'
df.to_csv(filename, index=False)